<a href="https://colab.research.google.com/github/sudheervobili/Pytorch/blob/main/Pytorch_model_pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import numpy as np

# Dummy Dataset: [Income, Loan Amount, Credit Score] --> Approved
data = np.array([
    [50000, 200000, 700, 1],
    [30000, 150000, 650, 0],
    [40000, 120000, 670, 1],
    [25000, 100000, 600, 0],
    [60000, 220000, 720, 1],
    [45000, 140000, 690, 1],
    [35000, 130000, 660, 0],
    [55000, 210000, 710, 1]
], dtype=np.float32)

X = data[:, :-1]  # Features
y = data[:, -1:]  # Target

# Feature Scaling (Very important for neural networks)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Train-Test Split
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.25, random_state=42)

# Convert to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32)

# Model using nn.Sequential
class HomeLoanModel(nn.Module):
    def __init__(self, num_features):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(num_features, 5),
            nn.ReLU(),
            nn.Linear(5, 4),
            nn.ReLU(),
            nn.Linear(4, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.model(x)

# Initialize model
model = HomeLoanModel(num_features=X_train.shape[1])

# Loss & Optimizer
criterion = nn.BCELoss()  # Binary classification
optimizer = optim.Adam(model.parameters(), lr=0.01)

# Training Loop
epochs = 500
for epoch in range(epochs):
    optimizer.zero_grad()
    outputs = model(X_train_tensor)
    loss = criterion(outputs, y_train_tensor)
    loss.backward()
    optimizer.step()

    if (epoch+1) % 50 == 0:
        print(f'Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}')

# Testing
with torch.no_grad():
    predictions = model(X_test_tensor)
    predicted = (predictions > 0.5).float()
    accuracy = (predicted.eq(y_test_tensor).sum().item()) / y_test_tensor.size(0)
    print("\nTest Predictions:", predicted.numpy().flatten())
    print("Test Accuracy:", accuracy * 100, "%")


Epoch [50/500], Loss: 0.6692
Epoch [100/500], Loss: 0.6421
Epoch [150/500], Loss: 0.6371
Epoch [200/500], Loss: 0.6365
Epoch [250/500], Loss: 0.6365
Epoch [300/500], Loss: 0.6365
Epoch [350/500], Loss: 0.6365
Epoch [400/500], Loss: 0.6365
Epoch [450/500], Loss: 0.6365
Epoch [500/500], Loss: 0.6365

Test Predictions: [1. 1.]
Test Accuracy: 50.0 %
